<a href="https://colab.research.google.com/github/BrunoGeraldine/B3_Prediction_2023/blob/main/notebook/C1_Linear_Regress_ScikitLearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn


In [ ]:
# Import Bibliotecas
import numpy  as np
import pandas as pd


from sklearn                   import datasets, linear_model
from sklearn.metrics           import mean_squared_error, r2_score
from sklearn.preprocessing     import MinMaxScaler
from sklearn.neural_network    import MLPRegressor
from sklearn.model_selection   import GridSearchCV
from sklearn.feature_selection import SelectKBest

%matplotlib inline

In [ ]:
## Help Functions

# Supress scientific Notation
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
# Load data
df = pd.read_parquet('/content/all_bovespa_17to23.parquet')

In [ ]:
df

In [ ]:
# Capturando a ação desejada
acao = 'VALE3'
df_acao = df[df['sigla_acao'] == acao]

In [ ]:
df_acao

In [ ]:
df_acao.dtypes

In [ ]:
df_acao.tail()

In [ ]:
# Criando as médias moveis
df_acao['MM5d'] = df_acao.loc[: ,('preco_fech')].rolling(5).mean()
df_acao['MM21d'] = df_acao.loc[: ,('preco_fech')].rolling(21).mean()


In [ ]:
df_acao.head()

In [ ]:
# Empurrando para trás ps valores das ações
df_acao['preco_fech'] = df_acao['preco_fech'].shift(-1)

df_acao.head()
""" Faço isso para prever o dia seguinte usando 
as features do dia anterior"""

In [ ]:
# Eliminando os dados nulos
df_acao.dropna(inplace=True)
df_acao
# Elimino aqui 21 linhas

In [ ]:
# Verificando quantidade de linhas
qtd_linha = len(df_acao)

qtd_linha_treino = qtd_linha - 390 #(70%)

qtd_linha_teste = qtd_linha - 13 #(1%)

qtd_linha_valid = qtd_linha_treino - qtd_linha_teste

info = (
    f'Linhas treino = 0: {qtd_linha_treino}',
    f'Linhas teste = {qtd_linha_treino}:{qtd_linha_teste}',
    f'Linhas validação = {qtd_linha_teste}:{qtd_linha}'

)

info

In [ ]:
#Reindexando o df
df_acao = df_acao.reset_index(drop=True)
df_acao

In [ ]:
#Separando features e labels
features = df_acao.drop(['sigla_acao', 'nome_acao', 'data_pregao', 'preco_fech', 'Year'], 1)
labels = df_acao['preco_fech']


In [ ]:
# escolhendo as melhores features utilizando o KBest
features_list = ('preco_abert', 'qtd_negocios', 'vol_negocios', 'MM5d', 'MM21d')

k_best_features = SelectKBest(k='all')
k_best_features.fit_transform(features, labels)
k_best_features_scores = k_best_features.scores_
raw_pairs = zip(features_list[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

k_best_features_final = dict(ordered_pairs[:15])
best_features = k_best_features_final.keys()

print('')
print('Melhores features')
print(k_best_features_final)

In [ ]:
features = df_acao.drop(['sigla_acao', 'nome_acao', 'data_pregao', 'preco_fech', 'preco_abert', 'Year', 'MM21d'], 1)

In [ ]:
# Normalizando os Dados usando MinMaxSacale
scaler = MinMaxScaler().fit(features)
features_scale = scaler.transform(features)

print('Features: ', features_scale.shape)
print(features_scale)

In [ ]:
# Separando em treino e test
X_train = features_scale[:qtd_linha_treino]
Y_train = 